In [ ]:
import inro.modeller as _m
import inro.emme.desktop as _d
import traceback as _traceback

import os
import re
import copy
import csv as csv

import sqlite3
import pandas as pd
import numpy as np

In [ ]:
eb = _m.Modeller().emmebank
util = _m.Modeller().tool("translink.util")
data_export = _m.Modeller().tool("translink.RTM3.stage4.dataexport")



In [ ]:
dt = _d.app.connect()
de = dt.data_explorer()
db = de.active_database()
ebs = de.databases()


In [ ]:
am_scenid = eb.scenario(int(eb.matrix("msAmScen").data))
md_scenid = eb.scenario(int(eb.matrix("msMdScen").data))
pm_scenid = eb.scenario(int(eb.matrix("msPmScen").data))

# following needs to be the scenario object
scens = [am_scenid,md_scenid,pm_scenid]

## Define the update function - this needs to be moved to the model at some point (done - not needed any more)

In [ ]:
def autoStats(eb):
    util = _m.Modeller().tool("translink.util")
    aonDist = util.get_matrix_numpy(eb, "mfdistAON").flatten()
    df = util.get_ijensem_df(eb, 'gy','gy')

    df['amSovDemand1'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_1_Am").flatten()
    df['amSovDemand2'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_2_Am").flatten()
    df['amSovDemand3'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_3_Am").flatten()
    df['amSovDemand4'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_4_Am").flatten()

    df['mdSovDemand1'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_1_Md").flatten()
    df['mdSovDemand2'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_2_Md").flatten()
    df['mdSovDemand3'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_3_Md").flatten()
    df['mdSovDemand4'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_4_Md").flatten()

    df['pmSovDemand1'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_1_Pm").flatten()
    df['pmSovDemand2'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_2_Pm").flatten()
    df['pmSovDemand3'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_3_Pm").flatten()
    df['pmSovDemand4'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_4_Pm").flatten()


    df['amHovDemand1'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_1_Am").flatten()
    df['amHovDemand2'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_2_Am").flatten()
    df['amHovDemand3'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_3_Am").flatten()
    df['amHovDemand4'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_4_Am").flatten()

    df['mdHovDemand1'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_1_Md").flatten()
    df['mdHovDemand2'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_2_Md").flatten()
    df['mdHovDemand3'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_3_Md").flatten()
    df['mdHovDemand4'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_4_Md").flatten()

    df['pmHovDemand1'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_1_Pm").flatten()
    df['pmHovDemand2'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_2_Pm").flatten()
    df['pmHovDemand3'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_3_Pm").flatten()
    df['pmHovDemand4'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_4_Pm").flatten()
    
    df['amLGVDemand1'] = util.get_matrix_numpy(eb, "mfLGVAM").flatten()
    df['amHGVDemand1'] = util.get_matrix_numpy(eb, "mfHGVAM").flatten()
    df['mdLGVDemand1'] = util.get_matrix_numpy(eb, "mfLGVMD").flatten()
    df['mdHGVDemand1'] = util.get_matrix_numpy(eb, "mfHGVMD").flatten()
    df['pmLGVDemand1'] = util.get_matrix_numpy(eb, "mfLGVPM").flatten()
    df['pmHGVDemand1'] = util.get_matrix_numpy(eb, "mfHGVPM").flatten()

    # create copy of trips and multiply by distance to get simple vkt
    df2 = df.drop(['gy_i','gy_j'], 1).mul(aonDist, axis = 0)
    df2.rename(columns= lambda x: re.sub('Demand', 'Vkt', x), inplace=True)
    # need to bring back ij refs for group by
    df2 = pd.concat([util.get_ijensem_df(eb, 'gy','gy'),df2], axis=1)
    df2 = pd.melt(df2, id_vars = ['gy_i','gy_j'], var_name = 'timeModeVot', value_name='vkt')
    df2Gy =  df2.groupby(['gy_i','gy_j','timeModeVot'])
    df2Gy = df2Gy.sum().reset_index()

    df3 = pd.melt(df, id_vars = ['gy_i','gy_j'], var_name = 'timeModeVot', value_name = 'trips')
    dfGy = df3.groupby(['gy_i','gy_j','timeModeVot'])
    dfGy = dfGy.sum().reset_index()

    # extract toll data
    dfToll = util.get_ijensem_df(eb, 'gy','gy')
    dfToll['amSovToll1'] = df['amSovDemand1'].multiply(util.get_matrix_numpy(eb, "mfAmSovTollVOT1").flatten(), axis = 0)
    dfToll['amSovToll2'] = df['amSovDemand2'].multiply(util.get_matrix_numpy(eb, "mfAmSovTollVOT2").flatten(), axis = 0)
    dfToll['amSovToll3'] = df['amSovDemand3'].multiply(util.get_matrix_numpy(eb, "mfAmSovTollVOT3").flatten(), axis = 0)
    dfToll['amSovToll4'] = df['amSovDemand4'].multiply(util.get_matrix_numpy(eb, "mfAmSovTollVOT4").flatten(), axis = 0)
    dfToll['mdSovToll1'] = df['mdSovDemand1'].multiply(util.get_matrix_numpy(eb, "mfMdSovTollVOT1").flatten(), axis = 0)
    dfToll['mdSovToll2'] = df['mdSovDemand2'].multiply(util.get_matrix_numpy(eb, "mfMdSovTollVOT2").flatten(), axis = 0)
    dfToll['mdSovToll3'] = df['mdSovDemand3'].multiply(util.get_matrix_numpy(eb, "mfMdSovTollVOT3").flatten(), axis = 0)
    dfToll['mdSovToll4'] = df['mdSovDemand4'].multiply(util.get_matrix_numpy(eb, "mfMdSovTollVOT4").flatten(), axis = 0)
    dfToll['pmSovToll1'] = df['pmSovDemand1'].multiply(util.get_matrix_numpy(eb, "mfPmSovTollVOT1").flatten(), axis = 0)
    dfToll['pmSovToll2'] = df['pmSovDemand2'].multiply(util.get_matrix_numpy(eb, "mfPmSovTollVOT2").flatten(), axis = 0)
    dfToll['pmSovToll3'] = df['pmSovDemand3'].multiply(util.get_matrix_numpy(eb, "mfPmSovTollVOT3").flatten(), axis = 0)
    dfToll['pmSovToll4'] = df['pmSovDemand4'].multiply(util.get_matrix_numpy(eb, "mfPmSovTollVOT4").flatten(), axis = 0)
    dfToll['amHovToll1'] = df['amHovDemand1'].multiply(util.get_matrix_numpy(eb, "mfAmHovTollVOT1").flatten(), axis = 0)
    dfToll['amHovToll2'] = df['amHovDemand2'].multiply(util.get_matrix_numpy(eb, "mfAmHovTollVOT2").flatten(), axis = 0)
    dfToll['amHovToll3'] = df['amHovDemand3'].multiply(util.get_matrix_numpy(eb, "mfAmHovTollVOT3").flatten(), axis = 0)
    dfToll['amHovToll4'] = df['amHovDemand4'].multiply(util.get_matrix_numpy(eb, "mfAmHovTollVOT4").flatten(), axis = 0)
    dfToll['mdHovToll1'] = df['mdHovDemand1'].multiply(util.get_matrix_numpy(eb, "mfMdHovTollVOT1").flatten(), axis = 0)
    dfToll['mdHovToll2'] = df['mdHovDemand2'].multiply(util.get_matrix_numpy(eb, "mfMdHovTollVOT2").flatten(), axis = 0)
    dfToll['mdHovToll3'] = df['mdHovDemand3'].multiply(util.get_matrix_numpy(eb, "mfMdHovTollVOT3").flatten(), axis = 0)
    dfToll['mdHovToll4'] = df['mdHovDemand4'].multiply(util.get_matrix_numpy(eb, "mfMdHovTollVOT4").flatten(), axis = 0)
    dfToll['pmHovToll1'] = df['pmHovDemand1'].multiply(util.get_matrix_numpy(eb, "mfPmHovTollVOT1").flatten(), axis = 0)
    dfToll['pmHovToll2'] = df['pmHovDemand2'].multiply(util.get_matrix_numpy(eb, "mfPmHovTollVOT2").flatten(), axis = 0)
    dfToll['pmHovToll3'] = df['pmHovDemand3'].multiply(util.get_matrix_numpy(eb, "mfPmHovTollVOT3").flatten(), axis = 0)
    dfToll['pmHovToll4'] = df['pmHovDemand4'].multiply(util.get_matrix_numpy(eb, "mfPmHovTollVOT4").flatten(), axis = 0)
    dfToll['amLGVToll1'] = df['amLGVDemand1'].multiply(util.get_matrix_numpy(eb, "mfAmLgvToll").flatten(), axis = 0)
    dfToll['amHGVToll1'] = df['amHGVDemand1'].multiply(util.get_matrix_numpy(eb, "mfAmHgvToll").flatten(), axis = 0)
    dfToll['mdLGVToll1'] = df['mdLGVDemand1'].multiply(util.get_matrix_numpy(eb, "mfMdLgvToll").flatten(), axis = 0)
    dfToll['mdHGVToll1'] = df['mdHGVDemand1'].multiply(util.get_matrix_numpy(eb, "mfMdHgvToll").flatten(), axis = 0)
    dfToll['pmLGVToll1'] = df['pmLGVDemand1'].multiply(util.get_matrix_numpy(eb, "mfPmLgvToll").flatten(), axis = 0)
    dfToll['pmHGVToll1'] = df['pmHGVDemand1'].multiply(util.get_matrix_numpy(eb, "mfPmHgvToll").flatten(), axis = 0)



    dfToll = pd.melt(dfToll, id_vars = ['gy_i','gy_j'], var_name = 'timeModeVot', value_name = 'tolls')
    dfTollGy = dfToll.groupby(['gy_i','gy_j','timeModeVot'])
    dfTollGy = dfTollGy.sum().reset_index()


    # create categorical fields from original colnames
    dfTimeModeVot = dfGy['timeModeVot'].str.extract(r'(?P<peak>[a|m|p]{1}[m|d])(?P<mode>Sov|Hov|LGV|HGV)Demand(?P<votclass>\d)')
    dfTimeModeVotT = dfTollGy['timeModeVot'].str.extract(r'(?P<peak>[a|m|p]{1}[m|d])(?P<mode>Sov|Hov|LGV|HGV)Toll(?P<votclass>\d)')

    dfGy = pd.concat([dfGy,dfTimeModeVot], axis=1)
    dfGy = dfGy[['gy_i','gy_j','peak','mode','votclass','trips']]

    df2Gy = pd.concat([df2Gy,dfTimeModeVot], axis=1)
    df2Gy = df2Gy[['gy_i','gy_j','peak','mode','votclass','vkt']]

    dfTollGy = pd.concat([dfTollGy,dfTimeModeVotT], axis=1)
    dfTollGy = dfTollGy[['gy_i','gy_j','peak','mode','votclass','tolls']]

    conn = util.get_db_byname(eb, 'trip_summaries.db')
    dfGy.to_sql(name='autoTripsGy', con=conn, flavor='sqlite', index=False, if_exists='replace')
    df2Gy.to_sql(name='autoVktGy', con=conn, flavor='sqlite', index=False, if_exists='replace')
    dfTollGy.to_sql(name='autoTollGy', con=conn, flavor='sqlite', index=False, if_exists='replace')
    conn.close()

## Data processing functions

In [ ]:
def ParamsTable(eb):
    params = {'autoOpCost' : eb.matrix('msautoOpCost').data,
              'lgvOpCost' : eb.matrix('mslgvOpCost').data,
              'hgvOpCost' : eb.matrix('mshgvOpCost').data,
              'lgvTollFac' : eb.matrix('mslgvTollFac').data,
              'hgvTollFac' : eb.matrix('mshgvTollFac').data,
              'sovTollFac' : eb.matrix('mssovTollFac').data,
              'hovTollFac' : eb.matrix('mshovTollFac').data,
             }

    params = pd.DataFrame(params, index=[0])   
    conn = util.get_db_byname(eb, 'trip_summaries.db')
    params.to_sql(name='params', con=conn, flavor='sqlite', index=False, if_exists='replace')
    conn.close()

In [ ]:
def TransitRevenue(eb):
    AmFareBus = util.get_matrix_numpy(eb, "AmBusFare").flatten()
    MdFareBus = util.get_matrix_numpy(eb, "MdBusFare").flatten()
    PmFareBus = util.get_matrix_numpy(eb, "PmBusFare").flatten()
    AmFareRail = util.get_matrix_numpy(eb, "AmRailFare").flatten()
    MdFareRail = util.get_matrix_numpy(eb, "MdRailFare").flatten()
    PmFareRail = util.get_matrix_numpy(eb, "PmRailFare").flatten()
    AmFareWce = util.get_matrix_numpy(eb, "AmWceFare").flatten()
    PmFareWce = util.get_matrix_numpy(eb, "PmWceFare").flatten()

    AmDemandBus = util.get_matrix_numpy(eb, "mfbusAm").flatten()
    MdDemandBus = util.get_matrix_numpy(eb, "mfbusMd").flatten()
    PmDemandBus = util.get_matrix_numpy(eb, "mfbusPm").flatten()
    AmDemandRail = util.get_matrix_numpy(eb, "mfrailAm").flatten()
    MdDemandRail = util.get_matrix_numpy(eb, "mfrailMd").flatten()
    PmDemandRail = util.get_matrix_numpy(eb, "mfrailPm").flatten()
    AmDemandWce = util.get_matrix_numpy(eb, "mfWCEAm").flatten()
    PmDemandWce = util.get_matrix_numpy(eb, "mfWCEPm").flatten()
    
    df = pd.DataFrame(index=[0])
    
    df['AmBusRevenue'] = (AmFareBus * AmDemandBus).sum()
    df['MdBusRevenue'] = (MdFareBus * MdDemandBus).sum()
    df['PmBusRevenue'] = (PmFareBus * PmDemandBus).sum()
    df['AmRailRevenue'] = (AmFareRail * AmDemandRail).sum()
    df['MdRailRevenue'] = (MdFareRail * MdDemandRail).sum()
    df['PmRailRevenue'] = (PmFareRail * PmDemandRail).sum()
    df['AmWceRevenue'] = (AmFareWce * AmDemandWce).sum()
    df['PmWceRevenue'] = (PmFareWce * PmDemandWce).sum()
    
    df = pd.melt(df)
    dfTimeMode = df['variable'].str.extract(r'(?P<peak>[A|M|P]{1}[m|d])(?P<mode>Bus|Rail|Wce)(?P<measure>Revenue)')
    df = pd.concat([df,dfTimeMode], axis=1)
    
    df = df[['peak','mode','measure','value']]
    
    conn = util.get_db_byname(eb, 'trip_summaries.db')
    df.to_sql(name='transitRevenue', con=conn, flavor='sqlite', index=False, if_exists='replace')
    conn.close()
    

In [ ]:
def LosDTime(eb):
    conn = util.get_db_byname(eb, 'trip_summaries.db')
    df = pd.read_sql("SELECT * FROM netResults", conn)

    df['posted_time'] = df['Length'] * 60 / df['posted_speed']
    df['losDtime'] = 0.0
    df.loc[df.vdf == 1, 'losDtime'] = df['posted_time']
    df.loc[df.vdf == 2, 'losDtime'] = df['Auto_Time']
    
    df.loc[df.vdf==25, 'losDtime'] = 0.25 + df['Length'] * 60 / df['posted_speed'] + 0.85 * (0.85) ** 4
    df.loc[df.vdf==35, 'losDtime'] = 0.25 + df['Length'] * 60 / df['posted_speed'] + 0.85 * (0.85) ** 4
    df.loc[df.vdf==45, 'losDtime'] = 0.25 + df['Length'] * 60 / df['posted_speed'] + 0.85 * (0.85) ** 4
    df.loc[df.vdf==55, 'losDtime'] = 0.25 + df['Length'] * 60 / df['posted_speed'] + 0.85 * (0.85) ** 4
    df.loc[df.vdf==65, 'losDtime'] = 0.25 + df['Length'] * 60 / df['posted_speed'] + 0.85 * (0.85) ** 4
    df.loc[df.vdf==75, 'losDtime'] = 0.25 + df['Length'] * 60 / df['posted_speed'] + 0.85 * (0.85) ** 4

    df.loc[df.vdf==85, 'losDtime'] = df['Length'] * 60 / df['posted_speed'] * (1+ 0.6 * 0.85 * (0.85) ** 5)
    df.loc[df.vdf==88, 'losDtime'] = df['Length'] * 60 / (df['posted_speed'] * 1.1)* (1 + 0.6 * 0.43 * (0.85) ** 5.25)

    df.loc[df.vdf==3, 'losDtime'] = df['Length'] * 60 / df['posted_speed'] + 0.85 * (0.85) ** 5
    df.loc[df.vdf==4, 'losDtime'] = df['Length'] * 60 / df['posted_speed'] + 0.85 * (0.85) ** 5
    df.loc[df.vdf==5, 'losDtime'] = df['Length'] * 60 / df['posted_speed'] + 0.85 * (0.85) ** 5
    df.loc[df.vdf==6, 'losDtime'] = df['Length'] * 60 / df['posted_speed'] + 0.85 * (0.85) ** 5
    df.loc[df.vdf==7, 'losDtime'] = df['Length'] * 60 / df['posted_speed'] + 0.85 * (0.85) ** 5
    
    df['losDspeed'] = df['Length'] * 60 / df['losDtime']
 
    df.to_sql(name='netResults', con=conn, flavor='sqlite', index=False, if_exists='replace')
    conn.close()

In [ ]:
def CalcParking(eb):
    util = _m.Modeller().tool("translink.util")
    df = util.get_ijensem_df(eb, 'gy','gy')

    ## Get Occupancy
    ## Get Cost Skims
    # Auto
    NoTAZ = len(util.get_matrix_numpy(eb, "zoneindex"))

    #######################
    # HBW
    #######################

    Occ = float(util.get_matrix_numpy(eb, 'HOVOccHbw'))
    Park_cost = util.get_matrix_numpy(eb, 'prk8hr')
    Park_cost = np.where(Park_cost>10, 10, Park_cost)
    Park_cost = Park_cost.reshape(1, NoTAZ) + np.zeros((NoTAZ, 1))

    # cost
    sovcst1 = 0.5*Park_cost
    sovcst2 = 0.5*Park_cost
    sovcst3 = 0.5*Park_cost

    hovcst1 = 0.5*Park_cost
    hovcst2 = 0.5*Park_cost
    hovcst3 = 0.5*Park_cost

    # demand
    sovdem1 = util.get_matrix_numpy(eb, "HbWSOVI1PerTrips")
    sovdem2 = util.get_matrix_numpy(eb, "HbWSOVI2PerTrips")
    sovdem3 = util.get_matrix_numpy(eb, "HbWSOVI3PerTrips")

    hv2dem1 = util.get_matrix_numpy(eb, "HbWHV2I1PerTrips")/2.0
    hv2dem2 = util.get_matrix_numpy(eb, "HbWHV2I2PerTrips")/2.0
    hv2dem3 = util.get_matrix_numpy(eb, "HbWHV2I3PerTrips")/2.0

    hv3dem1 = util.get_matrix_numpy(eb, "HbWHV3+I1PerTrips")/Occ
    hv3dem2 = util.get_matrix_numpy(eb, "HbWHV3+I2PerTrips")/Occ
    hv3dem3 = util.get_matrix_numpy(eb, "HbWHV3+I3PerTrips")/Occ

    hbwpark = ( sovcst1*sovdem1
                        + sovcst2*sovdem2
                        + sovcst3*sovdem3
                        + hovcst1*hv2dem1
                        + hovcst2*hv2dem2
                        + hovcst3*hv2dem3
                        + hovcst1*hv3dem1
                        + hovcst2*hv3dem2
                        + hovcst3*hv3dem3 )    

    #######################
    # HBU
    #######################

    Occ = float(util.get_matrix_numpy(eb, 'HOVOccHbu'))
    Park_cost = util.get_matrix_numpy(eb, 'prk8hr')
    Park_cost = np.where(Park_cost>10, 10, Park_cost)
    Park_cost = Park_cost.reshape(1, NoTAZ) + np.zeros((NoTAZ, 1))

    # cost
    sovcst = 0.5*Park_cost
    hovcst = 0.5*Park_cost

    ## Get demand
    sovdem = util.get_matrix_numpy(eb, "HbUSOVPerTrips")
    hovdem = util.get_matrix_numpy(eb, "HbUHOVPerTrips")/Occ

    hbupark = ( sovcst*sovdem
                    + hovcst*hovdem
                    )   

    #######################
    # HBSchool
    #######################

    # not clear we would have parking charges for school drop off

    #######################
    # HBShopping
    #######################

    Occ = float(util.get_matrix_numpy(eb, 'HOVOccHBshop'))
    Park_cost = util.get_matrix_numpy(eb, 'prk2hr')

    Park_cost = Park_cost.reshape(1, NoTAZ) + np.zeros((NoTAZ, 1))

    sovcst1 = 0.5*Park_cost
    sovcst2 = 0.5*Park_cost
    sovcst3 = 0.5*Park_cost

    hovcst1 = 0.5*Park_cost
    hovcst2 = 0.5*Park_cost
    hovcst3 = 0.5*Park_cost

    ## Get Demands
    sovdem1 = util.get_matrix_numpy(eb, "HbShSOVI1PerTrips")
    sovdem2 = util.get_matrix_numpy(eb, "HbShSOVI2PerTrips")
    sovdem3 = util.get_matrix_numpy(eb, "HbShSOVI3PerTrips")

    hovdem1 = util.get_matrix_numpy(eb, "HbShHOVI1PerTrips")/Occ
    hovdem2 = util.get_matrix_numpy(eb, "HbShHOVI2PerTrips")/Occ
    hovdem3 = util.get_matrix_numpy(eb, "HbShHOVI3PerTrips")/Occ


    # Calculate Costs

    hbshpark = ( sovcst1*sovdem1
                        + sovcst2*sovdem2
                        + sovcst3*sovdem3
                        + hovcst1*hovdem1
                        + hovcst2*hovdem2
                        + hovcst3*hovdem3
                        )    

    #######################
    # HBPerBus
    #######################

    Occ = float(util.get_matrix_numpy(eb, 'HOVOccHbpb'))
    Park_cost = util.get_matrix_numpy(eb, 'prk2hr')

    Park_cost = Park_cost.reshape(1, NoTAZ) + np.zeros((NoTAZ, 1))

    sovcst1 = 0.5*Park_cost
    sovcst2 = 0.5*Park_cost
    sovcst3 = 0.5*Park_cost

    hovcst1 = 0.5*Park_cost
    hovcst2 = 0.5*Park_cost
    hovcst3 = 0.5*Park_cost

    ## Get Demands

    sovdem1 = util.get_matrix_numpy(eb, "HbPbSOVI1PerTrips")
    sovdem2 = util.get_matrix_numpy(eb, "HbPbSOVI2PerTrips")
    sovdem3 = util.get_matrix_numpy(eb, "HbPbSOVI3PerTrips")

    hovdem1 = util.get_matrix_numpy(eb, "HbPbHOVI1PerTrips")/Occ
    hovdem2 = util.get_matrix_numpy(eb, "HbPbHOVI2PerTrips")/Occ
    hovdem3 = util.get_matrix_numpy(eb, "HbPbHOVI3PerTrips")/Occ

    # Calculate Costs

    hbpbpark = ( sovcst1*sovdem1
                        + sovcst2*sovdem2
                        + sovcst3*sovdem3
                        + hovcst1*hovdem1
                        + hovcst2*hovdem2
                        + hovcst3*hovdem3
                        )

    #######################
    # HBSocial
    #######################

    Occ = float(util.get_matrix_numpy(eb, 'HOVOccHBsoc'))
    Park_cost = util.get_matrix_numpy(eb, 'prk2hr')

    Park_cost = Park_cost.reshape(1, NoTAZ) + np.zeros((NoTAZ, 1))

    sovcst1 = 0.5*Park_cost
    sovcst2 = 0.5*Park_cost
    sovcst3 = 0.5*Park_cost

    hovcst1 = 0.5*Park_cost
    hovcst2 = 0.5*Park_cost
    hovcst3 = 0.5*Park_cost

    ## Get Demands

    sovdem1 = util.get_matrix_numpy(eb, "HbSoSOVI1PerTrips")
    sovdem2 = util.get_matrix_numpy(eb, "HbSoSOVI2PerTrips")
    sovdem3 = util.get_matrix_numpy(eb, "HbSoSOVI3PerTrips")

    hovdem1 = util.get_matrix_numpy(eb, "HbSoHOVI1PerTrips")/Occ
    hovdem2 = util.get_matrix_numpy(eb, "HbSoHOVI2PerTrips")/Occ
    hovdem3 = util.get_matrix_numpy(eb, "HbSoHOVI3PerTrips")/Occ

    # Calculate Costs

    hbsopark = ( sovcst1*sovdem1
                        + sovcst2*sovdem2
                        + sovcst3*sovdem3
                        + hovcst1*hovdem1
                        + hovcst2*hovdem2
                        + hovcst3*hovdem3
                        )

    #######################
    # HBEsc
    #######################

    # Not parking when escorting

    #######################
    # NHBWork
    #######################

    Occ = float(util.get_matrix_numpy(eb, 'HOVOccNHBw'))
    Park_cost = 0.5*(util.get_matrix_numpy(eb, 'prk8hr') +  util.get_matrix_numpy(eb, 'prk2hr'))
    Park_cost = np.where(Park_cost>10, 10, Park_cost)
    Park_cost = Park_cost.reshape(1, NoTAZ) + np.zeros((NoTAZ, 1))

    sovcst = Park_cost
    hovcst = Park_cost

    ## Get Demands

    sovdem = util.get_matrix_numpy(eb, "NHbWSOVPerTrips")
    hovdem = util.get_matrix_numpy(eb, "NHbWHOVPerTrips")/Occ

    # Calculate Costs

    nhbwpark = ( sovcst*sovdem
                        + hovcst*hovdem
                        )

    #######################
    # NHBOther
    #######################

    Occ = float(util.get_matrix_numpy(eb, 'HOVOccNHBo'))
    Park_cost = util.get_matrix_numpy(eb, 'prk2hr')
    Park_cost = Park_cost.reshape(1, NoTAZ) + np.zeros((NoTAZ, 1))

    hovcst = Park_cost

    ## Get Demands

    sovdem = util.get_matrix_numpy(eb, "NHbOSOVPerTrips")
    hovdem = util.get_matrix_numpy(eb, "NHbOHOVPerTrips")/Occ

    # Calculate Costs

    nhbopark = ( hovcst*sovdem
                        + hovcst*hovdem
                        )

    tot_parking = hbwpark.flatten() + hbupark.flatten() + hbshpark.flatten() + hbpbpark.flatten() + hbsopark.flatten() + nhbwpark.flatten() + nhbopark.flatten()
    df['parking_paid'] = tot_parking
 
    dfGy = df.groupby(['gy_i','gy_j'])
    dfGy = dfGy.sum().reset_index()

    conn = util.get_db_byname(eb, 'trip_summaries.db')
    dfGy.to_sql(name='parkingPaid', con=conn, flavor='sqlite', index=False, if_exists='replace')   
    conn.close()

In [ ]:
def ExecuteQry(eb, output_db, qry_file_name, measure_name=None, wide_to_long=False):
    output_db_name = output_db
    qry = qry_file_name
    
    proj_path = os.path.dirname(_m.Modeller().desktop.project.path)
    file_path = os.path.join(proj_path, 'Scripts\\tl_internal_tools\\summary_sql','{}'.format(qry))
 
    fd = open(file_path, 'r')
    sqlCmd = fd.read()
    fd.close()

    conn = util.get_db_byname(eb, 'trip_summaries.db')
    df = pd.read_sql(sqlCmd, conn)
    conn.close()

    if wide_to_long:
        df = pd.melt(df, id_vars = ['scenario','alternative','horizon_year','peak'], var_name = 'mode', value_name = 'value')
        df['measure'] = "{}".format(measure_name)
        df = df[['scenario','alternative','horizon_year','peak','mode','measure','value']]
    
    try:
        output_db = os.path.join(proj_path, output_db_name)
        conn = sqlite3.connect(output_db)
        df.to_sql(name='summary_metrics', con=conn, flavor='sqlite', index=False, if_exists='append')
        conn.close()

    except Exception as e:
        print("Query {} addition to summary table failed.".format(qry), e)

## Update all runs in project (this section probably not needed anymore)

In [ ]:
# THIS WILL BE DEPRECATED  with new model runs
for eb in ebs:
    title = eb.title()
        
    if title == 'Minimal Base Databank':
        continue
    
    eb.open()
    eb = _m.Modeller().emmebank
    try:
        data_export(eb)
        autoStats(eb)
        md = {'scenario' :'BusinessAsUsual',
              'alternative' : title,
              'horizon_year' : 2030}
        md = pd.DataFrame(md, index=[0])
        md['alternative'] = title
        conn = util.get_rtm_db(eb)
        md.to_sql(name='metadata', con=conn, flavor='sqlite', index=False, if_exists='replace')
        conn.close()
        conn = util.get_db_byname(eb, 'trip_summaries.db')
        md.to_sql(name='metadata', con=conn, flavor='sqlite', index=False, if_exists='replace')
        conn.close()      
        ParamsTable(eb)
        TransitRevenue(eb)
        LosDTime(eb)          
    
    except Exception as e:
        print("Scenario {} update failed.".format(title), e)

## Run the dataprocessing functions on all of the dbs

In [ ]:
# need to setup the loops

In [ ]:
for eb in ebs:
    title = eb.title()
        
    if title == 'Minimal Base Databank':
        continue
    
    eb.open()
    eb = _m.Modeller().emmebank
    try:    
        # ParamsTable(eb) # bring this back once the other stuff is standard output
        # TransitRevenue(eb)
        # LosDTime(eb)
        # CalcParking(eb)
        ExecuteQry(eb, 'mp_t1_bl.db', 'aggNetResults.sql')
        ExecuteQry(eb, 'mp_t1_bl.db', 'auto_tolls_by_mode_peak.sql')
        ExecuteQry(eb, 'mp_t1_bl.db', 'auto_trips_by_mode_peak.sql')
        ExecuteQry(eb, 'mp_t1_bl.db', 'avgTL_by_mode_peak.sql')
        ExecuteQry(eb, 'mp_t1_bl.db', 'congested_vht_by_mode_peak.sql', 'congested_vht', wide_to_long=True)
        ExecuteQry(eb, 'mp_t1_bl.db', 'congested_vkt_by_mode_peak.sql', 'congested_vkt', wide_to_long=True)
        ExecuteQry(eb, 'mp_t1_bl.db', 'daily_mode_share.sql')
        ExecuteQry(eb, 'mp_t1_bl.db', 'opcost_by_mode_peak.sql', 'opcost', wide_to_long=True)
        ExecuteQry(eb, 'mp_t1_bl.db', 'revenue_transit_by_mode_peak.sql')
        ExecuteQry(eb, 'mp_t1_bl.db', 'time_lost_by_mode_peak.sql', 'minutes_lost',wide_to_long=True)
        ExecuteQry(eb, 'mp_t1_bl.db', 'transit_ridership_daily.sql')
        ExecuteQry(eb, 'mp_t1_bl.db', 'transit_ridership_peak.sql')
        ExecuteQry(eb, 'mp_t1_bl.db', 'vht_by_mode_peak.sql', 'vht', wide_to_long=True)
        ExecuteQry(eb, 'mp_t1_bl.db', 'vkt_by_mode_peak.sql', 'vkt', wide_to_long=True)
        # ExecuteQry(eb, 'mp_t1_bl.db', 'daily_parking_paid.sql')       
    
    except Exception as e:
        print("Scenario {} update failed.".format(title), e)

## Add some additional data

In [ ]:
proj_path = os.path.dirname(_m.Modeller().desktop.project.path)
file_path = os.path.join(proj_path, 'Scripts\\tl_internal_tools\\expansion_factors.csv')

dfe = pd.read_csv(file_path)
output_db = os.path.join(proj_path, 'mp_t1_bl.db')
conn = sqlite3.connect(output_db)
dfe.to_sql(name='expansion_factors', con=conn, flavor='sqlite', index=False, if_exists='replace')
conn.close()


## Tools

In [ ]:
counter = 0
for eb in ebs:
    print counter, eb.title()
    counter += 1

In [ ]:
ebs[0].open()

In [ ]:
eb = _m.Modeller().emmebank

In [ ]:
title = eb.title()

In [ ]:
print eb.title()

In [ ]:
conn = util.get_db_byname(eb, 'trip_summaries.db')
df = pd.read_sql("SELECT * FROM netResults", conn)
conn.close()

In [ ]:
proj_path = os.path.dirname(_m.Modeller().desktop.project.path)
# file_path = os.path.join(proj_path, 'Scripts\\tl_internal_tools\\expansion_factors.csv')

output_db = os.path.join(proj_path, 'mp_t1.db')
conn = sqlite3.connect(output_db)
df.to_sql(name='net_results_bl_2016', con=conn, flavor='sqlite', index=False, if_exists='replace')
conn.close()

In [ ]:
sql = """ 
SELECT 
    e.i
    ,e.j
    ,e.peakperiod

    ,e.speed as speed2016
    ,f.speed as speed2045

    ,e.losDspeed
    ,CASE
        WHEN (e.speed / e.losDspeed) < 0.85  THEN 1 ELSE 0 END cong_check_2016
            
    ,CASE 
        WHEN (f.speed / f.losDspeed) < 0.85  THEN 1 ELSE 0 END cong_check_2045   

    ,CASE 
        WHEN (f.speed / f.losDspeed) < 0.85 and (e.speed / e.losDspeed) >= 0.85 THEN 1 ELSE 0 END new_future_congestion   


  FROM net_results_bl_2016 e
    LEFT JOIN net_results_bl_2045 f on f.i = e.i and f.j = e.j and f.peakperiod = e.peakperiod
"""

In [ ]:
proj_path = os.path.dirname(_m.Modeller().desktop.project.path)
output_db = os.path.join(proj_path, 'mp_t1.db')

conn = sqlite3.connect(output_db)
df = pd.read_sql(sql, conn)
conn.close()

## Create Input Data for Heat Map Plots 

In [ ]:
util = _m.Modeller().tool("translink.util")
df = pd.concat([util.get_pd_ij_df(eb),util.get_ijensem_df(eb, 'gy','gy')], axis=1)

df['amSovDemand1'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_1_Am").flatten()
df['amSovDemand2'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_2_Am").flatten()
df['amSovDemand3'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_3_Am").flatten()
df['amSovDemand4'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_4_Am").flatten()

df['mdSovDemand1'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_1_Md").flatten()
df['mdSovDemand2'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_2_Md").flatten()
df['mdSovDemand3'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_3_Md").flatten()
df['mdSovDemand4'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_4_Md").flatten()

df['pmSovDemand1'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_1_Pm").flatten()
df['pmSovDemand2'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_2_Pm").flatten()
df['pmSovDemand3'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_3_Pm").flatten()
df['pmSovDemand4'] = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_4_Pm").flatten()


df['amHovDemand1'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_1_Am").flatten()
df['amHovDemand2'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_2_Am").flatten()
df['amHovDemand3'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_3_Am").flatten()
df['amHovDemand4'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_4_Am").flatten()

df['mdHovDemand1'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_1_Md").flatten()
df['mdHovDemand2'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_2_Md").flatten()
df['mdHovDemand3'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_3_Md").flatten()
df['mdHovDemand4'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_4_Md").flatten()

df['pmHovDemand1'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_1_Pm").flatten()
df['pmHovDemand2'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_2_Pm").flatten()
df['pmHovDemand3'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_3_Pm").flatten()
df['pmHovDemand4'] = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_4_Pm").flatten()

df['amLGVDemand1'] = util.get_matrix_numpy(eb, "mfLGVAM").flatten()
df['amHGVDemand1'] = util.get_matrix_numpy(eb, "mfHGVAM").flatten()
df['mdLGVDemand1'] = util.get_matrix_numpy(eb, "mfLGVMD").flatten()
df['mdHGVDemand1'] = util.get_matrix_numpy(eb, "mfHGVMD").flatten()
df['pmLGVDemand1'] = util.get_matrix_numpy(eb, "mfLGVPM").flatten()
df['pmHGVDemand1'] = util.get_matrix_numpy(eb, "mfHGVPM").flatten()

In [ ]:
dfTime = pd.concat([util.get_pd_ij_df(eb),util.get_ijensem_df(eb, 'gy','gy')], axis=1)
dfTime['amSovTime1'] = df['amSovDemand1'].multiply(util.get_matrix_numpy(eb, "mfAmSovTimeVOT1").flatten(), axis = 0)
dfTime['amSovTime2'] = df['amSovDemand2'].multiply(util.get_matrix_numpy(eb, "mfAmSovTimeVOT2").flatten(), axis = 0)
dfTime['amSovTime3'] = df['amSovDemand3'].multiply(util.get_matrix_numpy(eb, "mfAmSovTimeVOT3").flatten(), axis = 0)
dfTime['amSovTime4'] = df['amSovDemand4'].multiply(util.get_matrix_numpy(eb, "mfAmSovTimeVOT4").flatten(), axis = 0)
dfTime['mdSovTime1'] = df['mdSovDemand1'].multiply(util.get_matrix_numpy(eb, "mfMdSovTimeVOT1").flatten(), axis = 0)
dfTime['mdSovTime2'] = df['mdSovDemand2'].multiply(util.get_matrix_numpy(eb, "mfMdSovTimeVOT2").flatten(), axis = 0)
dfTime['mdSovTime3'] = df['mdSovDemand3'].multiply(util.get_matrix_numpy(eb, "mfMdSovTimeVOT3").flatten(), axis = 0)
dfTime['mdSovTime4'] = df['mdSovDemand4'].multiply(util.get_matrix_numpy(eb, "mfMdSovTimeVOT4").flatten(), axis = 0)
dfTime['pmSovTime1'] = df['pmSovDemand1'].multiply(util.get_matrix_numpy(eb, "mfPmSovTimeVOT1").flatten(), axis = 0)
dfTime['pmSovTime2'] = df['pmSovDemand2'].multiply(util.get_matrix_numpy(eb, "mfPmSovTimeVOT2").flatten(), axis = 0)
dfTime['pmSovTime3'] = df['pmSovDemand3'].multiply(util.get_matrix_numpy(eb, "mfPmSovTimeVOT3").flatten(), axis = 0)
dfTime['pmSovTime4'] = df['pmSovDemand4'].multiply(util.get_matrix_numpy(eb, "mfPmSovTimeVOT4").flatten(), axis = 0)
dfTime['amHovTime1'] = df['amHovDemand1'].multiply(util.get_matrix_numpy(eb, "mfAmHovTimeVOT1").flatten(), axis = 0)
dfTime['amHovTime2'] = df['amHovDemand2'].multiply(util.get_matrix_numpy(eb, "mfAmHovTimeVOT2").flatten(), axis = 0)
dfTime['amHovTime3'] = df['amHovDemand3'].multiply(util.get_matrix_numpy(eb, "mfAmHovTimeVOT3").flatten(), axis = 0)
dfTime['amHovTime4'] = df['amHovDemand4'].multiply(util.get_matrix_numpy(eb, "mfAmHovTimeVOT4").flatten(), axis = 0)
dfTime['mdHovTime1'] = df['mdHovDemand1'].multiply(util.get_matrix_numpy(eb, "mfMdHovTimeVOT1").flatten(), axis = 0)
dfTime['mdHovTime2'] = df['mdHovDemand2'].multiply(util.get_matrix_numpy(eb, "mfMdHovTimeVOT2").flatten(), axis = 0)
dfTime['mdHovTime3'] = df['mdHovDemand3'].multiply(util.get_matrix_numpy(eb, "mfMdHovTimeVOT3").flatten(), axis = 0)
dfTime['mdHovTime4'] = df['mdHovDemand4'].multiply(util.get_matrix_numpy(eb, "mfMdHovTimeVOT4").flatten(), axis = 0)
dfTime['pmHovTime1'] = df['pmHovDemand1'].multiply(util.get_matrix_numpy(eb, "mfPmHovTimeVOT1").flatten(), axis = 0)
dfTime['pmHovTime2'] = df['pmHovDemand2'].multiply(util.get_matrix_numpy(eb, "mfPmHovTimeVOT2").flatten(), axis = 0)
dfTime['pmHovTime3'] = df['pmHovDemand3'].multiply(util.get_matrix_numpy(eb, "mfPmHovTimeVOT3").flatten(), axis = 0)
dfTime['pmHovTime4'] = df['pmHovDemand4'].multiply(util.get_matrix_numpy(eb, "mfPmHovTimeVOT4").flatten(), axis = 0)
dfTime['amLGVTime1'] = df['amLGVDemand1'].multiply(util.get_matrix_numpy(eb, "mfAmLgvTime").flatten(), axis = 0)
dfTime['amHGVTime1'] = df['amHGVDemand1'].multiply(util.get_matrix_numpy(eb, "mfAmHgvTime").flatten(), axis = 0)
dfTime['mdLGVTime1'] = df['mdLGVDemand1'].multiply(util.get_matrix_numpy(eb, "mfMdLgvTime").flatten(), axis = 0)
dfTime['mdHGVTime1'] = df['mdHGVDemand1'].multiply(util.get_matrix_numpy(eb, "mfMdHgvTime").flatten(), axis = 0)
dfTime['pmLGVTime1'] = df['pmLGVDemand1'].multiply(util.get_matrix_numpy(eb, "mfPmLgvTime").flatten(), axis = 0)
dfTime['pmHGVTime1'] = df['pmHGVDemand1'].multiply(util.get_matrix_numpy(eb, "mfPmHgvTime").flatten(), axis = 0)

In [ ]:
df = pd.melt(df, id_vars = ['i','j','gy_i','gy_j'], var_name = 'timeModeVot', value_name = 'trips')

In [ ]:
dfTimeModeVot = df['timeModeVot'].str.extract(r'(?P<peak>[a|m|p]{1}[m|d])(?P<mode>Sov|Hov|LGV|HGV)Demand(?P<votclass>\d)')

In [ ]:
df = pd.concat([dfTimeModeVot, df], axis = 1)

In [ ]:
del dfTimeModeVot

In [ ]:
dfTime = pd.melt(dfTime, id_vars = ['i','j','gy_i','gy_j'], var_name = 'timeModeVot', value_name = 'total_mins')

In [ ]:
df['total_mins'] = dfTime['total_mins']

In [ ]:
del dfTime

In [ ]:
df['scenario'] = 'mp_t1_hwy'

In [ ]:
df['mode'] = np.where(df['mode'].isin(['Sov','Hov']), 'auto', 'truck')

In [ ]:
df = df.groupby(['scenario','peak','mode','i', 'j', 'gy_i', 'gy_j'])
df = df.sum().reset_index()

In [ ]:
df.columns

In [ ]:
df = df[['scenario','peak', 'mode', 'i', 'j', 'gy_i', 'gy_j', 'trips', 'total_mins']]

In [ ]:
proj_path = os.path.dirname(_m.Modeller().desktop.project.path)
file_path = os.path.join(proj_path, 'hwy.txt')

In [ ]:
df.to_csv(file_path, sep = '\t', index=False)